# Final Project: Music Genre Clustering

In [297]:
# the following packages can be used to extract features from audio data
from pyAudioAnalysis import audioBasicIO as abio
from pyAudioAnalysis import ShortTermFeatures as stf
from pyAudioAnalysis import aud
from pydub import AudioSegment
import librosa
# you may use other packages too.
import os,glob,eyed3
import pandas as pd
import numpy as np


from sklearn import cluster


pd.set_option('display.max_columns', None)


Please use the given data (including the sound tracks and an additional feature file) to cluster the sound tracks. You may use any methods for feature engineering, but the clustering methods should be limited to the methods in this class. Do not hand label the tracks. Then generate the submission.json file for submission purposes. 

In [296]:
cw = os.getcwd()
os.listdir(cw)

data = cw + '/data'
fils = os.listdir(data)
fils = tuple(fils)

In [121]:
feats = pd.read_csv('feature_set.csv',header=[0,1,2])
feats.set_index(feats.columns[0])

chroma_cens                                \
                                 kurtosis                                 
                                       01        02        03        04   
(feature, statistics, number)                                             
0                               -0.371332  0.033316  0.280583  1.221006   
1                                2.431309 -0.701034 -0.599058 -0.723412   
2                                1.938045  0.291241  1.248885 -0.056932   
3                                1.974653 -0.038609 -0.047155 -0.590124   
4                               -0.648081 -0.278046 -0.520639 -0.739611   
...                                   ...       ...       ...       ...   
85                               0.009023  0.299104  0.026383 -0.706047   
86                               0.204764 -0.833226 -0.919060 -0.935129   
87                              -1.034451  0.245840 -0.405058  0.246232   
88                              -1.000784 -0.163925  0.159248 -0.185568   
89                              -0.701113 -0.086331 -0.948332 -0.833377   

                                                                       \
                                                                        
                                     05        06        07        08   
(feature, statistics, number)                                           
0                              0.371610 -0.587465 -0.942494 -1.002133   
1                             -0.218545 -0.035764  0.302542  0.267308   
2                             -0.434033 -0.158136 -0.206080  0.075225   
3                              0.729950 -0.805870 -0.795872 -0.002026   
4                             -0.597592  0.610723 -0.406698  0.031583   
...                                 ...       ...       ...       ...   
85                            -0.295000  0.020247 -0.383799 -0.089401   
86                            -0.254850  0.701667 -1.101793 -1.015899   
87                            -0.576097 -0.480972 -1.117766  0.475257   
88                             0.588360 -0.404090  0.666972  0.817534   
89                            -0.728301  0.519564  0.205120 -0.842308   

                                                                       \
                                                                        
                                     09        10        11        12   
(feature, statistics, number)                                           
0                              1.700059  0.133831 -0.989182 -0.617790   
1                              0.422103 -0.051525  0.268163  0.455092   
2                             -0.656558  0.132843 -0.573856  2.503376   
3                             -0.517309 -0.849433 -0.921587  3.064209   
4                             -0.253484 -0.050135  0.695594 -0.754319   
...                                 ...       ...       ...       ...   
85                            -0.135325  0.651826 -0.537588 -0.774158   
86                            -1.168807  1.059893 -0.200050  2.158769   
87                            -1.103677 -0.125222 -0.794122 -1.243121   
88                            -0.891178 -0.849103  3.533587  0.539222   
89                            -1.125524 -0.139352 -0.963818 -1.108550   

                                                                       \
                                    max                                 
                                     01        02        03        04   
(feature, statistics, number)                                           
0                              0.686195  0.642466  0.660841  0.622677   
1                              0.721947  0.512168  0.545834  0.584512   
2                              0.659792  0.599739  0.687972  0.502656   
3                              0.693670  0.640284  0.712163  0.535815   
4                              0.653399  0.563665  0.539257  0.519335   
...                                 ...       ...      

In [151]:
rows = feats.columns.tolist()
feature = [i[0] for i in rows][1:]
statistics = [i[1] for i in rows][1:]

# unique column name values
feature_set = list(set(feature))
statistics_set = list(set(statistics))

print("Sets of headers:")
print(feature_set)
print()
print(statistics_set)

Sets of headers:
['chroma_cqt', 'spectral_bandwidth', 'zcr', 'spectral_contrast', 'spectral_rolloff', 'spectral_centroid', 'mfcc', 'tonnetz', 'chroma_stft', 'chroma_cens', 'rmse']

['kurtosis', 'skew', 'std', 'min', 'median', 'mean', 'max']


In [299]:
#read in and enable mp3 play
AudioSegment.from_file("data/2.mp3")

In [298]:
#https://gitlab.cs.dartmouth.edu/cy/show_and_tell/blob/d8dae9870895e1afe431360db729e2de597799b6/emotion/pyAudioAnalysis-master/audioBasicIO.py
[Fs,x] = audioBasicIO.read_audio_file("data/0.mp3")

In [259]:
#This function converts the input signal (stored in a numpy array) to MONO (if it is STEREO)
audioBasicIO.stereo_to_mono(x)

array([   0. ,    0. ,    0. , ..., 1803.5, 1445.5, 1327. ])

In [294]:
def convertDirMP3ToWav(dirName, Fs, nC, useMp3TagsAsName = False):
    '''
    This function converts the MP3 files stored in a folder to WAV. If required, the output names of the WAV files are based on MP3 tags, otherwise the same names are used.
    ARGUMENTS:
     - dirName:     the path of the folder where the MP3s are stored
     - Fs:          the sampling rate of the generated WAV files
     - nC:          the number of channesl of the generated WAV files
     - useMp3TagsAsName:    True if the WAV filename is generated on MP3 tags
    '''

    types = (dirName+os.sep+'*.mp3',) # the tuple of file types
    print(types)
    filesToProcess = [] 

    for files in types:
        filesToProcess.extend(glob.glob(files))     

    for f in filesToProcess:
        #tag.link(f)
        audioFile = eyed3.load(f)     
        #print(audioFile)
        if useMp3TagsAsName and audioFile.tag != None:          
            artist = audioFile.tag.artist
            title = audioFile.tag.title
            if artist!=None and title!=None:
                if len(title)>0 and len(artist)>0:
                    wavFileName = ntpath.split(f)[0] + os.sep + artist.replace(","," ") + " --- " + title.replace(","," ") + ".wav"
                else:
                    wavFileName = f.replace(".mp3",".wav")  
            else:
                wavFileName = f.replace(".mp3",".wav")                      
        else:
            wavFileName = f.replace(".mp3",".wav")      
        command = "avconv -i \"" + f + "\" -ar " +str(Fs) + " -ac " + str(nC) + " \"" + wavFileName + "\"";
        print (command)
        #os.system(command.encode('ascii','ignore').replace("\0",""))


In [295]:
convertDirMP3ToWav("data",44.1,1)

('data/*.mp3',)
avconv -i "data/82.mp3" -ar 44.1 -ac 1 "data/82.wav"
avconv -i "data/69.mp3" -ar 44.1 -ac 1 "data/69.wav"
avconv -i "data/41.mp3" -ar 44.1 -ac 1 "data/41.wav"
avconv -i "data/55.mp3" -ar 44.1 -ac 1 "data/55.wav"
avconv -i "data/7.mp3" -ar 44.1 -ac 1 "data/7.wav"
avconv -i "data/54.mp3" -ar 44.1 -ac 1 "data/54.wav"
avconv -i "data/6.mp3" -ar 44.1 -ac 1 "data/6.wav"
avconv -i "data/40.mp3" -ar 44.1 -ac 1 "data/40.wav"
avconv -i "data/68.mp3" -ar 44.1 -ac 1 "data/68.wav"
avconv -i "data/83.mp3" -ar 44.1 -ac 1 "data/83.wav"
avconv -i "data/81.mp3" -ar 44.1 -ac 1 "data/81.wav"
avconv -i "data/4.mp3" -ar 44.1 -ac 1 "data/4.wav"
avconv -i "data/56.mp3" -ar 44.1 -ac 1 "data/56.wav"
avconv -i "data/42.mp3" -ar 44.1 -ac 1 "data/42.wav"
avconv -i "data/43.mp3" -ar 44.1 -ac 1 "data/43.wav"
avconv -i "data/5.mp3" -ar 44.1 -ac 1 "data/5.wav"
avconv -i "data/57.mp3" -ar 44.1 -ac 1 "data/57.wav"
avconv -i "data/80.mp3" -ar 44.1 -ac 1 "data/80.wav"
avconv -i "data/84.mp3" -ar 44.1 -ac 1

## Submission to leaderboard

In [ ]:
# put your clustering result in labels
# if you have k clusters, the labels should be a vector of elements taking values
# from 0~ k-1

import json

submission = {}

# change this to the url, which I will hand out later.
submission['url'] = "http://127.0.0.1:8384"

# change this to your group id, which I will hand out later.
submission['vsID'] = " " 

submission['number_of_clusters'] = len(set(labels ))
submission['labels'] = []
for x in labels:
    submission['labels'].append( int(x) )

with open('submission.json', 'w') as outfile:
    json.dump(submission, outfile)


I will provide a submission tutorial session early next week. 